# 🎛️ Interactive Price Prediction App (Gradio + Modal)

This notebook launches an interactive **Gradio web app** that lets you enter a product description and receive a predicted price in real-time.  

It connects to our **fine-tuned LLaMA 3.1 model**, deployed on **Modal**, for fast and efficient inference. The model runs remotely on GPU, while the app interface runs in your Colab notebook or browser.

> 💡 Great for demos, testing, or sharing with others!


In [ ]:
# 📦 Install required libraries
!pip install -q modal gradio

In [ ]:
# 🔌 Import required packages
import modal
import gradio as gr
from google.colab import userdata
import os

In [ ]:
# 🔐 Set up Modal authentication using Colab secrets
modal_token_id = userdata.get("MODAL_TOKEN_ID")
modal_token_secret = userdata.get("MODAL_TOKEN_SECRET")

# Set environment variables for Modal CLI to use
os.environ["MODAL_TOKEN_ID"] = modal_token_id
os.environ["MODAL_TOKEN_SECRET"] = modal_token_secret

### 🔐 Authenticate with Modal

Before running the deployment or calling your Modal function, make sure you’ve added your `MODAL_TOKEN_ID` and `MODAL_TOKEN_SECRET` as notebook secrets.

> 💡 You can do this by clicking the 🔑 key icon on the left panel in Google Colab and adding:
> - `MODAL_TOKEN_ID`: your Modal token ID (starts with `ak-`)
> - `MODAL_TOKEN_SECRET`: your Modal secret key (starts with `sk-`)

Then, run the following command in a code cell to log in:


In [ ]:
!modal token set --token-id {modal_token_id} --token-secret {modal_token_secret}

In [ ]:
# 🔍 Get the deployed class
pricer_cls = modal.Cls.from_name("pricer-electronics-service", "Pricer")
# Instantiate it
pricer = pricer_cls()

In [ ]:
# 🧠 Define the interface function
def predict_price(description):
    try:
        price = pricer.price.remote(description)
        return f"💸 Estimated Price: ${price:.2f}"
    except Exception as e:
        return f"⚠️ Error: {str(e)}"


In [ ]:
# 🎛️ Create a clean, full-width Gradio app with a centered title
with gr.Blocks(title="The Price is Right") as demo: # fill_width=True
    gr.Markdown(
        """
        <div style="text-align: center;">
            <h1>🔍 Electronics Price Predictor</h1>
            <p>Enter a product description below to estimate its price using the fine-tuned LLaMA-3.1-8B model.</p>
        </div>
        """
    )

    with gr.Row():
        input_text = gr.Textbox(
            label="📝 Product Description",
            placeholder="e.g. Wireless earbuds with noise cancellation",
            lines=3,
            scale=1
        )
    with gr.Row():
        output = gr.Textbox(label="💰 Predicted Price")


    with gr.Row():
        predict_button = gr.Button("🚀 Estimate Price")

    predict_button.click(fn=predict_price, inputs=input_text, outputs=output)

# 🚀 Launch the app
demo.launch()
